# Script to compute the kinematics (for the first time)

specifically: we compute knee, hip, elbow and arm angle with rtmo, and the ankle angle with body26 (since rtmo has no foot keypoint)

eventually, we store the data to be able to load it directly without recomputing joint angles

In [107]:
import json
from pathlib import Path

import biopsykit as bp
import matplotlib.pyplot as plt
import pandas as pd
import pingouin as pg
import seaborn as sns
from biopsykit.questionnaires.utils import compute_scores, wide_to_long
from biopsykit.utils.dataframe_handling import convert_nan
from fau_colors import cmaps, register_fausans_font
from scipy.interpolate import interp1d

from stressgait_analysis.dataset import StressGaitDataset

from src.stressgait_analysis.gait_helper_functions import compute_leg_kinematics, compute_arm_kinematics

%matplotlib widget
%load_ext autoreload
%autoreload 2
from stressgait_analysis.gait_helper_functions import compute_HS, resample
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [108]:
deploy_type = "local"

config_dict = json.load(Path("../../config.json").open(encoding="utf-8"))

base_path = Path(config_dict[deploy_type]["base_path"])
gait_data_path = Path(config_dict[deploy_type]["gait_data_path"])

In [109]:
dataset = StressGaitDataset(base_path, coarse_condition=True, gait_data_path=gait_data_path, specify_bouts=True, specify_speed=True)

In [110]:

data = []
for trial in dataset:

    keypoints = trial.load_keypoint_trajectories()
    hs_events = compute_HS(keypoints)
    n_cycle = 0
    #iterate over individual cycles:
    for i in range(len(hs_events)-1):
        cycle_start = hs_events[i]
        #print(cycle_start)
        cycle_end = hs_events[i+1]
        leg_kinematics = compute_leg_kinematics(keypoints[cycle_start:cycle_end])
        arm_kinematics = compute_arm_kinematics(keypoints[cycle_start:cycle_end])

        #resample to 1900
        leg_kinematics = resample(leg_kinematics, n=101)
        arm_kinematics = resample(arm_kinematics, n=101)

        data.extend([
             (*trial.index.values[0], n_cycle, i, *leg_row, *arm_row)
                for i, (leg_row, arm_row) in enumerate(zip(leg_kinematics.values, arm_kinematics.values))
                ])

In [111]:
kinematics_df = pd.DataFrame(data, columns=[*dataset.index.columns, "cycle_idx", "percentage_of_stride", *leg_kinematics.columns, *arm_kinematics.columns],)
kinematics_df.set_index([*dataset.index.columns, "cycle_idx", "percentage_of_stride"], inplace=True)

In [112]:
kinematics_df

hip_flexion  \
participant condition bout speed cycle_idx percentage_of_stride                
VP_01       omc       1    slow  0         0                       17.179218   
                                           1                       16.507507   
                                           2                       15.723399   
                                           3                       14.842070   
                                           4                       13.871923   
...                                                                      ...   
VP_47       omc       2    fast  0         96                      20.071597   
                                           97                      20.253160   
                                           98                      20.278922   
                                           99                      20.043864   
                                           100                     19.750845   

                                                                 knee_flexion  \
participant condition bout speed cycle_idx percentage_of_stride                 
VP_01       omc       1    slow  0         0                        12.116574   
                                           1                        11.065104   
                                           2                        10.305527   
                                           3                         9.732018   
                                           4                         9.200402   
...                                                                       ...   
VP_47       omc       2    fast  0         96                       10.679031   
                                           97                        9.500543   
                                           98                        8.550138   
                                           99                        8.147652   
                                           100                       7.866927   

                                                                 ankle_plantarflexion  \
participant condition bout speed cycle_idx percentage_of_stride                         
VP_01       omc       1    slow  0         0                                      NaN   
                                           1                                      NaN   
                                           2                                      NaN   
                                           3                                      NaN   
                                           4                                      NaN   
...                                                                               ...   
VP_47       omc       2    fast  0         96                                     NaN   
                                           97                                     NaN   
                                           98                                     NaN   
                                           99                                     NaN   
                                           100                                    NaN   

                                                                 elbow_flexion_angles  \
participant condition bout speed cycle_idx percentage_of_stride                         
VP_01       omc       1    slow  0         0                                71.313000   
                                           1                                71.139635   
                                           2                                70.885186   
                                           3                                70.569000   
                                           4                                70.212616   
...                                                                               ...   
VP_47       omc       2    fast  0         96                               11.163907   
           

In [117]:
data = []
for trial in dataset:
    print(trial.index.participant)
    keypoints = trial.load_keypoint_trajectories(model="body26")
    hs_events = compute_HS(keypoints)
    n_cycle = 0
    #iterate over individual cycles:
    for i in range(len(hs_events)-1):
        cycle_start = hs_events[i]
        #print(cycle_start)
        cycle_end = hs_events[i+1]
        leg_kinematics = compute_leg_kinematics(keypoints[cycle_start:cycle_end])

        #resample to 1900
        leg_kinematics = resample(leg_kinematics, n=101)

        data.extend([
             (*trial.index.values[0], n_cycle, i, *leg_row,)
                for i, leg_row in enumerate(leg_kinematics.values)
                ])

0    VP_01
Name: participant, dtype: object
0    VP_01
Name: participant, dtype: object
0    VP_02
Name: participant, dtype: object
0    VP_02
Name: participant, dtype: object
0    VP_05
Name: participant, dtype: object
0    VP_05
Name: participant, dtype: object
0    VP_06
Name: participant, dtype: object
0    VP_06
Name: participant, dtype: object
0    VP_07
Name: participant, dtype: object
0    VP_07
Name: participant, dtype: object
0    VP_08
Name: participant, dtype: object
0    VP_08
Name: participant, dtype: object
0    VP_10
Name: participant, dtype: object
0    VP_10
Name: participant, dtype: object
0    VP_11
Name: participant, dtype: object
0    VP_11
Name: participant, dtype: object
0    VP_12
Name: participant, dtype: object
0    VP_12
Name: participant, dtype: object
0    VP_13
Name: participant, dtype: object
0    VP_13
Name: participant, dtype: object
0    VP_15
Name: participant, dtype: object
0    VP_15
Name: participant, dtype: object
0    VP_16
Name: participant, dt

StopIteration: 

In [54]:
kinematics_df_ankle = pd.DataFrame(data, columns=[*dataset.index.columns, "cycle_idx", "percentage_of_stride", *leg_kinematics.columns],)

kinematics_df.loc[:,"ankle_plantarflexion"] = kinematics_df_ankle.loc[:,"ankle_plantarflexion"].values

In [56]:
kinematics_df

hip_flexion  \
participant condition bout speed cycle_idx percentage_of_stride                
VP_01       omc       1    slow  0         0                       17.179218   
                                           1                       16.507507   
                                           2                       15.723399   
                                           3                       14.842070   
                                           4                       13.871923   
...                                                                      ...   
VP_02       control   1    slow  0         96                      21.739787   
                                           97                      20.736279   
                                           98                      19.576186   
                                           99                      18.590380   
                                           100                     17.691718   

                                                                 knee_flexion  \
participant condition bout speed cycle_idx percentage_of_stride                 
VP_01       omc       1    slow  0         0                        12.116574   
                                           1                        11.065104   
                                           2                        10.305527   
                                           3                         9.732018   
                                           4                         9.200402   
...                                                                       ...   
VP_02       control   1    slow  0         96                       11.032843   
                                           97                        7.691451   
                                           98                        4.944496   
                                           99                        3.064177   
                                           100                       1.617176   

                                                                 ankle_plantarflexion  \
participant condition bout speed cycle_idx percentage_of_stride                         
VP_01       omc       1    slow  0         0                               -11.059877   
                                           1                                -9.919752   
                                           2                                -8.658710   
                                           3                                -7.326190   
                                           4                                -5.992229   
...                                                                               ...   
VP_02       control   1    slow  0         96                                2.424773   
                                           97                                2.680475   
                                           98                                2.666419   
                                           99                                2.363001   
                                           100                               1.914900   

                                                                 elbow_flexion_angles  \
participant condition bout speed cycle_idx percentage_of_stride                         
VP_01       omc       1    slow  0         0                                71.313000   
                                           1                                71.139635   
                                           2                                70.885186   
                                           3                                70.569000   
                                           4                                70.212616   
...                                                                               ...   
VP_02       control   1    slow  0         96                               24.863728   
           

## store the data

In [118]:
filename = dataset.base_path.joinpath("kinematics/kinematics.pkl")
kinematics_df.to_pickle(filename)


# Great! Kinematics can now be loaded directly from the dataset!


In [119]:
metadata_df = dataset.get_subset(participant="VP_01").index
metadata_df

,participant,condition,bout,speed
0,VP_01,omc,1,slow
1,VP_01,omc,2,fast
